# 2021 June Embed

## Imports

In [ ]:
import sys
import delphi.plotter as dp
from delphi.cpp.DelphiPython import AnalysisGraph, InitialBeta, InitialDerivative
import pandas as pd
import numpy as np
#import graphviz
#from IPython.display import display
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

## Read Data

In [ ]:
df_temp = pd.read_csv('../../data/mini_use_case/TerraClimateOromiaMonthlyMaxTemp.csv')
temperature = df_temp['(deg C) Max Temperature (TerraClimate) at State, 1958-01-01 to 2019-12-31'].tolist()[:72]

df_rain = pd.read_csv('../../data/mini_use_case/TerraClimateOromiaMontlhyPrecip.csv')
rain = df_rain['(mm) Precipitation (TerraClimate) at State, 1958-01-01 to 2019-12-31'].tolist()[:72]

## Create Base CAG

In [ ]:
statements = [
    (
        ("", 1, "rain"),
        ("", -1, "temperature"),
    )
]

data = {"rain": ("Monthly Precipitation (mm)", rain),
        "temperature": ("Monthly Max Temperature (F)", temperature)
        }
G = AnalysisGraph.from_causal_fragments_with_data((statements, data), kde_kernels=1000)

G.to_png('2021_june_embed.png', rankdir="TB", simplified_labels=False)
Image('2021_june_embed.png')

## Train Model
### Head node modeling related parameteres
<ol>
<li>concept_periods</li>
    <ul>
        <li>Period for each head concept.</li>
        <li>Default is 1.</li>
        <li>A dictionary: {'concept name': period}</li>
    </ul>
<li>concept_center_measures</li>
    <ul>
        <li>How to calculate the central tendency for each head concept.</li>
        <li>mean or median.</li>
        <li>Default is median.</li>
    </ul>

<li>concept_models</li>
    <ul>
        <li>The model to be used for the head concept.</li>
        <li>The choises are:</li>
            <ol>
                <li>center - Always predicts the center value for each partition.</li>
                <--li>absolute_change - Predicts based on the absolute change between adjacent partiton centers. $ac_{i} = center_{i+1} - center_i$</li-->
                <--li>relative_change - Predicts based on the absolute change between adjacent partiton centers.</li-->
            </ol>
    </ul>

</ol>

In [ ]:
def train_predict_plot(G, period, center, model, constraints={}, concept_min_vals=-10000, concept_max_vals=1000):
    # Training
    G.run_train_model(model_id='june_embed'
                      res=70,
                      burn=0,
                      initial_beta=InitialBeta.ZERO,
                      initial_derivative=InitialDerivative.DERI_ZERO,
                      use_heuristic=False,
                      use_continuous=False,
                      train_start_timestep=0,
                      train_timesteps=48,
                      concept_periods={'rain': period},
                      concept_center_measures={'rain': center},  # mean, median
                      concept_models={'rain': model},  # center, absolute_change, relative_change
                      concept_min_vals={'rain': concept_min_vals},
                      concept_max_vals={'rain': concept_max_vals}
                      )

    # Predicting
    G.generate_prediction(49, 23, constraints=constraints)

    # Plotting
    model_state = G.get_complete_state()
    dp.delphi_plotter(model_state, num_bins=400, rotation=45,
                out_dir='plots', file_name_prefix='', save_csv=False)

#### Using median

In [ ]:
train_predict_plot(G, period=12, center='median', model='center')
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')

#### Using mean

In [ ]:
train_predict_plot(G, period=12, center='mean', model='center')
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')

#### Let us try a different period

In [ ]:
train_predict_plot(G, period=24, center='median', model='center')
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')

### Constraints

<p>Three model options, center, absolute_change and relative_change behave differently upon constraints.</p>

#### center with constraints

In [ ]:
train_predict_plot(G, period=12, center='median', model='center', constraints={12: [('rain', 'Monthly Precipitation (mm)', 120)]})
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')

### Guiding the modeler with bounds

In [ ]:
train_predict_plot(G, period=12, center='median', model='absolute_change', constraints={7: [('rain', 'Monthly Precipitation (mm)', 15)]})
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')

In [ ]:
train_predict_plot(G, period=12, center='median', model='absolute_change', constraints={7: [('rain', 'Monthly Precipitation (mm)', 15)]}, concept_min_vals=0)
Image('plots/8_Predictions_Median_and_CI_Monthly Precipitation (mm).png')